In [2]:
#
# A script to fetch weather data from NASA POWER and OpenWeatherMap,
# process it, and save it as both a DSSAT-compatible .WTH file and a .CSV file.
#

import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime
from collections import defaultdict
from typing import Optional, Dict

def get_o3(api_key: str, lat: float, lon: float, start_str: str, end_str: str) -> pd.DataFrame:
    """Fetches O3 data from OpenWeatherMap API."""
    if not api_key or api_key == "YOUR_API_KEY_HERE":
        print("Warning: OpenWeatherMap API key is missing. Proceeding without O3 data.")
        return pd.DataFrame()
    
    print("Fetching OpenWeatherMap O3 data...")
    try:
        start_ts = int(pd.to_datetime(start_str).timestamp())
        end_ts = int(pd.to_datetime(end_str).replace(hour=23, minute=59).timestamp())
        api_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start_ts}&end={end_ts}&appid={api_key}"
        
        response = requests.get(api_url, timeout=30)
        response.raise_for_status()
        data = response.json().get('list', [])

        if not data:
            print("...O3 data fetched, but no records returned for the period.")
            return pd.DataFrame()

        daily_data = defaultdict(list)
        for entry in data:
            local_date = datetime.fromtimestamp(entry['dt']).date()
            daily_data[local_date].append(entry['components'].get('o3', 0))

        records = [{'date': date, 'ozon7': np.mean(vals)} for date, vals in sorted(daily_data.items())]
        df = pd.DataFrame(records)
        df['date'] = pd.to_datetime(df['date'])
        print("...O3 data fetched successfully.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching O3 data: {e}. Proceeding without it.")
        return pd.DataFrame()

def _fetch_nasa_weather(lat: float, lon: float, start: str, end: str) -> pd.DataFrame:
    """Fetches NASA POWER weather data."""
    print("Fetching NASA POWER data...")
    try:
        url = "https://power.larc.nasa.gov/api/temporal/daily/point"
        params = {
            "start": pd.to_datetime(start).strftime("%Y%m%d"),
            "end": pd.to_datetime(end).strftime("%Y%m%d"),
            "latitude": lat, "longitude": lon, "community": "AG",
            "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2MDEW,PRECTOTCORR,WS2M,RH2M,EVPTRNS",
            "format": "JSON"
        }
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        data = response.json()["properties"]["parameter"]
        df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
        df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
        df = df.replace([-999, -99], np.nan)
        df = df.rename(columns={
            "ALLSKY_SFC_SW_DWN": "srad", "T2M_MAX": "tmax", "T2M_MIN": "tmin",
            "T2MDEW": "dewp", "PRECTOTCORR": "rain", "WS2M": "wind",
            "RH2M": "rhum", "EVPTRNS": "evap"
        })
        print("...NASA data fetched successfully.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"FATAL: Error fetching NASA POWER data: {e}")
        return pd.DataFrame()

def create_weather_file(
    api_key: str,
    lat: float,
    lon: float,
    start_date: str,
    end_date: str,
    site_name: str = "SITE",
    insi: str = "XXXX",
    elev: Optional[float] = None
) -> Optional[pd.DataFrame]:
    """
    Fetches weather data, writes a standard DSSAT-compatible .WTH file,
    and saves the data to a .CSV file.
    """
    weather_df = _fetch_nasa_weather(lat, lon, start_date, end_date)
    if weather_df.empty:
        print("Could not fetch base weather data from NASA POWER. Aborting.")
        return None

    o3_df = get_o3(api_key, lat, lon, start_date, end_date)
    
    df = weather_df
    if not o3_df.empty:
        df = pd.merge(df, o3_df, on='date', how='left')

    if 'ozon7' in df.columns:
        df['ozon7'] = df['ozon7'].fillna(50.0)
    else:
        df['ozon7'] = 50.0
    
    df['dco2'] = 421.0

    df["wind"] = df["wind"] * 86.4
    df["month"] = df["date"].dt.month
    monthly_srad_avg = df.groupby("month")["srad"].mean().to_dict()
    df["par"] = df["month"].map(monthly_srad_avg)
    df["vapr"] = 0.6108 * np.exp((17.27 * df["dewp"]) / (df["dewp"] + 237.3))
    
    df["YRDOY"] = df["date"].dt.strftime("%y%j")
    
    tav = ((df["tmax"] + df["tmin"]) / 2).mean()
    amp = (df["tmax"].mean() - df["tmin"].mean())

    df_out = df.fillna(-99)
    
    output_cols = ["YRDOY","srad","tmax","tmin","rain","rhum","wind","dewp","par","vapr","dco2","ozon7","evap"]
    
    output_dir = "weather"
    os.makedirs(output_dir, exist_ok=True)
    
    year_2_digit = pd.to_datetime(start_date).strftime('%y')
    wth_file = os.path.join(output_dir, f"{insi}{year_2_digit}01.WTH")
    print(f"\nWriting weather data to: {wth_file}")

    with open(wth_file, "w") as f:
        f.write(f"*WEATHER DATA : {site_name},NASA\n\n")
        f.write("@ INSI      LAT     LONG    ELEV   TAV   AMP REFHT WNDHT   CCO2\n")
        f.write(f"  {insi:<4s}  {lat:7.3f} {lon:7.3f} {elev if elev else -99:5.1f} {tav:5.1f} {amp:5.1f}   2.0   2.0 {df['dco2'].mean():6.1f}\n\n")
        f.write("@DATE  SRAD  TMAX  TMIN  RAIN  RHUM  WIND  DEWP   PAR  VAPR   DCO2  OZON7  EVAP\n")
        
        for _, row in df_out[output_cols].iterrows():
            f.write(f"{int(row.YRDOY):7d} {row.srad:5.1f} {row.tmax:5.1f} {row.tmin:5.1f} "
                      f"{row.rain:5.1f} {row.rhum:5.1f} {row.wind:5.1f} {row.dewp:5.1f} "
                      f"{row.par:6.1f} {row.vapr:5.2f} {row.dco2:6.1f} {int(row.ozon7):6d} {row.evap:5.1f}\n")
    
    print("✅ WTH File created successfully.")

    # Save the dataframe as a CSV file
    csv_file = wth_file.replace('.WTH', '.csv')
    print(f"Writing weather data to CSV: {csv_file}")
    
    csv_df = df_out[output_cols].copy()
    csv_df.to_csv(csv_file, index=False, float_format='%.3f')
    
    print("✅ CSV file created successfully.")
    
    return df_out

if __name__ == "__main__":
    # ======================================================================
    # USER CONFIGURATION - PLEASE EDIT THE VALUES BELOW
    # ======================================================================
    
    # Get your free API key from https://openweathermap.org/api/air-pollution
    API_KEY = "YOUR_API_KEY_HERE"
    
    # Location details
    LATITUDE = 13.0827  # Latitude for Chennai, India
    LONGITUDE = 80.2707 # Longitude for Chennai, India
    ELEVATION = 6.0     # Elevation in meters
    SITE_NAME = "CHENNAI"
    INSI = "CHEN"       # 4-character identifier for the weather station
    
    # Date range for the weather data
    START_DATE = "2024-01-01"
    END_DATE = "2024-12-31"
    
    # ======================================================================
    
    print("--- Starting Weather File Generation ---")
    
    # Call the main function to fetch data and create the files
    final_data = create_weather_file(
        api_key=API_KEY,
        lat=LATITUDE,
        lon=LONGITUDE,
        start_date=START_DATE,
        end_date=END_DATE,
        site_name=SITE_NAME,
        insi=INSI,
        elev=ELEVATION
    )
    
    if final_data is not None:
        print("\n--- Process Complete ---")
        # Optional: display the first 5 rows of the final dataframe
        # print("\nFirst 5 rows of data:")
        # print(final_data.head())
    else:
        print("\n--- Process Failed ---")

--- Starting Weather File Generation ---
Fetching NASA POWER data...
...NASA data fetched successfully.

Writing weather data to: weather\CHEN2401.WTH
✅ WTH File created successfully.
Writing weather data to CSV: weather\CHEN2401.csv
✅ CSV file created successfully.

--- Process Complete ---


In [10]:
"""
A script to fetch weather data, process it, and save it as a .CSV file
with a standard date column (YYYY-MM-DD) and embedded metadata.

---------------------------------
PARAMETER EXPLANATIONS
---------------------------------

1. Metadata Header Parameters (in comments at the top of the CSV)
-------------------------------------------------------------------
These parameters describe the location and general climate characteristics.

- insi (Station Identifier): A unique 4-character code for the weather site (e.g., 'ITHY').
- lat (Latitude): Geographic latitude in decimal degrees (e.g., 17.530).
- long (Longitude): Geographic longitude in decimal degrees (e.g., 78.270).
- elev (Elevation): Altitude of the site above sea level in meters (m).
- tav (Average Annual Air Temperature): The average daily temperature over the data period, in degrees Celsius (degC).
- amp (Annual Amplitude of Temperature): A measure of temperature seasonality, calculated from the average TMAX and TMIN, in degrees Celsius (degC).
- refht (Reference Height): The height at which temperature/humidity are measured, in meters (m). Standard is 2.0.
- wndht (Wind Height): The height at which wind speed is measured, in meters (m).

2. Daily Weather Data Columns (in the main body of the CSV)
-------------------------------------------------------------
- date: The specific date for the row of data in YYYY-MM-DD format.
- srad (Solar Radiation): Total incoming solar energy. Unit: Megajoules per square meter per day (MJ/m^2/day).
- tmax (Maximum Temperature): Highest air temperature for the day. Unit: Degrees Celsius (degC).
- tmin (Minimum Temperature): Lowest air temperature for the day. Unit: Degrees Celsius (degC).
- rain (Precipitation): Total rainfall for the day. Unit: Millimeters (mm).
- rhum (Relative Humidity): Average relative humidity. Unit: Percent (%).
- wind (Wind Speed): Average wind speed. Unit: Kilometers per day (km/day).
- dewp (Dew Point Temperature): The temperature at which air becomes saturated. Unit: Degrees Celsius (degC).
- par (Photosynthetically Active Radiation): The portion of solar radiation used for photosynthesis. Unit: Megajoules per square meter per day (MJ/m^2/day).
- vapr (Vapor Pressure): The pressure exerted by water vapor in the air. Unit: Kilopascals (kPa).
- dco2 (Daily Atmospheric CO2): Concentration of carbon dioxide. Unit: Parts Per Million (ppm).
- ozon7 (Ozone): Concentration of ground-level ozone. Unit: Micrograms per cubic meter (ug/m^3).
- evap (Evaporation/Evapotranspiration): Potential water loss from the surface. Unit: Millimeters (mm).
"""

import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime
from collections import defaultdict
from typing import Optional

def get_o3(api_key: str, lat: float, lon: float, start_str: str, end_str: str) -> pd.DataFrame:
    """Fetches O3 data from OpenWeatherMap API."""
    if not api_key or api_key == "YOUR_API_KEY_HERE":
        print("Warning: OpenWeatherMap API key is missing. Proceeding without O3 data.")
        return pd.DataFrame()
    
    print("Fetching OpenWeatherMap O3 data...")
    try:
        start_ts = int(pd.to_datetime(start_str).timestamp())
        end_ts = int(pd.to_datetime(end_str).replace(hour=23, minute=59).timestamp())
        api_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start_ts}&end={end_ts}&appid={api_key}"
        
        response = requests.get(api_url, timeout=30)
        response.raise_for_status()
        data = response.json().get('list', [])

        if not data:
            print("...O3 data fetched, but no records returned for the period.")
            return pd.DataFrame()

        daily_data = defaultdict(list)
        for entry in data:
            local_date = datetime.fromtimestamp(entry['dt']).date()
            daily_data[local_date].append(entry['components'].get('o3', 0))

        records = [{'date': date, 'ozon7': np.mean(vals)} for date, vals in sorted(daily_data.items())]
        df = pd.DataFrame(records)
        df['date'] = pd.to_datetime(df['date'])
        print("...O3 data fetched successfully.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching O3 data: {e}. Proceeding without it.")
        return pd.DataFrame()

def _fetch_nasa_weather(lat: float, lon: float, start: str, end: str) -> pd.DataFrame:
    """Fetches NASA POWER weather data."""
    print("Fetching NASA POWER data...")
    try:
        url = "https://power.larc.nasa.gov/api/temporal/daily/point"
        params = {
            "start": pd.to_datetime(start).strftime("%Y%m%d"),
            "end": pd.to_datetime(end).strftime("%Y%m%d"),
            "latitude": lat, "longitude": lon, "community": "AG",
            "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2MDEW,PRECTOTCORR,WS2M,RH2M,EVPTRNS",
            "format": "JSON"
        }
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        data = response.json()["properties"]["parameter"]
        df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
        df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
        df = df.replace([-999, -99], np.nan)
        df = df.rename(columns={
            "ALLSKY_SFC_SW_DWN": "srad", "T2M_MAX": "tmax", "T2M_MIN": "tmin",
            "T2MDEW": "dewp", "PRECTOTCORR": "rain", "WS2M": "wind",
            "RH2M": "rhum", "EVPTRNS": "evap"
        })
        print("...NASA data fetched successfully.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"FATAL: Error fetching NASA POWER data: {e}")
        return pd.DataFrame()

def create_weather_csv(
    api_key: str,
    lat: float,
    lon: float,
    start_date: str,
    end_date: str,
    insi: str = "XXXX",
    elev: Optional[float] = 0.0,
    refht: float = 2.0,
    wndht: float = 3.0
) -> Optional[pd.DataFrame]:
    """
    Fetches weather data and saves it to a .CSV file with metadata comments.
    """
    weather_df = _fetch_nasa_weather(lat, lon, start_date, end_date)
    if weather_df.empty:
        print("Could not fetch base weather data from NASA POWER. Aborting.")
        return None

    o3_df = get_o3(api_key, lat, lon, start_date, end_date)
    
    df = weather_df
    if not o3_df.empty:
        df = pd.merge(df, o3_df, on='date', how='left')

    # Fill missing values and add derived columns
    if 'ozon7' in df.columns:
        df['ozon7'] = df['ozon7'].fillna(50.0)
    else:
        df['ozon7'] = 50.0
    
    df['dco2'] = 421.0
    df["wind"] = df["wind"] * 86.4
    df["month"] = df["date"].dt.month
    monthly_srad_avg = df.groupby("month")["srad"].mean().to_dict()
    df["par"] = df["month"].map(monthly_srad_avg)
    df["vapr"] = 0.6108 * np.exp((17.27 * df["dewp"]) / (df["dewp"] + 237.3))
    
    # Calculate metadata values
    tav = ((df["tmax"] + df["tmin"]) / 2).mean()
    amp = (df["tmax"].mean() - df["tmin"].mean())

    df_out = df.fillna(-99)
    
    output_cols = ["date", "srad", "tmax", "tmin", "rain", "rhum", "wind", "dewp", "par", "vapr", "dco2", "ozon7", "evap"]
    
    # --- CSV FILE CREATION ---
    output_dir = "weather"
    os.makedirs(output_dir, exist_ok=True)
    
    year_2_digit = pd.to_datetime(start_date).strftime('%y')
    csv_filename = os.path.join(output_dir, f"{insi}{year_2_digit}01.csv")
    print(f"\nWriting weather data to: {csv_filename}")

    # Prepare the header comments with metadata
    header_comments = (
        f"# insi={insi}\n"
        f"# lat={lat:.5f}\n"
        f"# long={lon:.5f}\n"
        f"# elev={elev:.1f}\n"
        f"# tav={tav:.1f}\n"
        f"# amp={amp:.1f}\n"
        f"# refht={refht:.1f}\n"
        f"# wndht={wndht:.1f}\n"
    )

    # Write the header and then the dataframe to the CSV file
    with open(csv_filename, 'w', newline='') as f:
        f.write(header_comments)
        df_out[output_cols].to_csv(f, index=False, float_format='%.3f', date_format='%Y-%m-%d')
    
    print("✅ CSV file created successfully.")
    
    return df_out

if __name__ == "__main__":
    # ======================================================================
    # USER CONFIGURATION - PLEASE EDIT THE VALUES BELOW
    # ======================================================================
    
    # Get your free API key from https://openweathermap.org/api/air-pollution
    API_KEY = "eddbc7c0c9e63e225934add809624c6e"
    
    # Date range for the weather data
    START_DATE = "2024-01-01"
    END_DATE = "2024-12-31"
    
    # Location and site parameters
    INSI = 'CHEN'
    LATITUDE = 17.530
    LONGITUDE = 78.270
    ELEVATION = 0.0
    REFHT = 2.0
    WNDHT = 3.0
    
    # ======================================================================
    
    print("--- Starting Weather CSV File Generation ---")
    
    final_data = create_weather_csv(
        api_key=API_KEY,
        lat=LATITUDE,
        lon=LONGITUDE,
        start_date=START_DATE,
        end_date=END_DATE,
        insi=INSI,
        elev=ELEVATION,
        refht=REFHT,
        wndht=WNDHT
    )
    
    if final_data is not None:
        print("\n--- Process Complete ---")
    else:
        print("\n--- Process Failed ---")

--- Starting Weather CSV File Generation ---
Fetching NASA POWER data...
...NASA data fetched successfully.
Fetching OpenWeatherMap O3 data...
...O3 data fetched successfully.

Writing weather data to: weather\CHEN2401.csv
✅ CSV file created successfully.

--- Process Complete ---


In [ ]:
meta_data = {}
with open(WEATHER_FILE_PATH, "r") as f:
    for line in f:
        if line.startswith("#"):
            comment = line.strip()[1:].split("=")
            key = comment[0].strip()
            value = comment[1].strip()

            # Try converting to int/float if possible
            if value.isdigit():
                value = int(value)
            else:
                try:
                    value = float(value)
                except ValueError:
                    pass  # keep as string

            meta_data[key] = value  

print("Meta Data found:", meta_data)

# Now read data ignoring comments
data = pd.read_csv(WEATHER_FILE_PATH, comment="#")
data["date"] = pd.to_datetime(data.date)
weather_station = WeatherStation(
    insi=meta_data['insi'], lat=meta_data['lat'], long=meta_data['long'],
    elev=meta_data['elev'], tav=meta_data["tav"], amp=meta_data["amp"],
    refht=meta_data["refht"], wndht=meta_data["wndht"],
    table=data
)   
weather_station